In [1]:
import pandas as pd
import requests
import json
import pickle

In [29]:
path = 'C:/Users/erihoward/Documents/GitHub/Heart_Disease_Classification/data/processed/male_test_features.csv'
data = pd.read_csv(path)
data['age'] = data['age'].astype('int32')
data['chest_pain_type'] = data['chest_pain_type'].astype('category')
data['resting_bp'] = data['resting_bp'].astype('int32')
data['cholesterol'] = data['cholesterol'].astype('int32')
data['fasting_blood_sugar'] = data['fasting_blood_sugar'].astype('category')
data['resting_ecg'] = data['resting_ecg'].astype('category')
data['max_heart_rate'] = data['max_heart_rate'].astype('int32')
data['exercise_angina'] = data['exercise_angina'].astype('category')
data['oldpeak'] = data['oldpeak'].astype('int32')
data['st_slope'] = data['st_slope'].astype('category')
data.dtypes

age                       int32
chest_pain_type        category
resting_bp                int32
cholesterol               int32
fasting_blood_sugar    category
resting_ecg            category
max_heart_rate            int32
exercise_angina        category
oldpeak                   int32
st_slope               category
dtype: object

In [30]:
# grab first 10 rows of data
test = data.head(10)

# load encoder and scaler
encoder = pickle.load(open('../models/pipelines/male_encoder.pkl', 'rb'))
scaler = pickle.load(open('../models/pipelines/male_scaler.pkl', 'rb'))

# encode and scale data
categorical_cols = test.select_dtypes(include='category').columns
test_encoded = pd.DataFrame(encoder.transform(test[categorical_cols]))
test_encoded.columns = encoder.get_feature_names_out(categorical_cols)
test = pd.concat([test.drop(categorical_cols, axis=1), test_encoded], axis=1)

numerical_cols = test.select_dtypes(include=['int32', 'int64', 'float64']).columns
test[numerical_cols] = scaler.transform(test[numerical_cols])

# convert to values so that I can convert to json and send to API
test_as_list = test.values.tolist()

In [35]:
test_as_list

[[0.36734693877551017,
  0.7000000000000001,
  0.5600814663951119,
  0.7394366197183099,
  0.2857142857142857,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0],
 [0.4897959183673468,
  0.64,
  0.41547861507128303,
  0.6760563380281691,
  0.42857142857142855,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [0.5306122448979591,
  0.8,
  0.3971486761710794,
  0.49295774647887325,
  0.42857142857142855,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 [0.6326530612244898,
  0.6,
  0.0,
  0.38732394366197187,
  0.2857142857142857,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [0.5918367346938775,
  0.75,
  0.3421588594704684,
  0.8028169014084506,
  0.42857142857142855,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
 

In [36]:
input = {'data': test_as_list}

In [37]:
response = requests.post('http://127.0.0.1:8000/predict', json=json.dumps(input))

In [38]:
json.loads(response.text)

{'predictions': [[0.9989005327224731, 0.0013097821502014995],
  [0.4971688687801361, 0.49967095255851746],
  [0.9828523993492126, 0.017741700634360313],
  [0.0023727957159280777, 0.997553288936615],
  [0.8831038475036621, 0.12016801536083221],
  [1.235544914379716e-06, 0.9999988079071045],
  [0.007781641092151403, 0.9919077157974243],
  [0.0022210427559912205, 0.9976906776428223],
  [1.8425556845613755e-05, 0.9999810457229614],
  [0.9994329810142517, 0.0007893249858170748]]}